In [2]:
import numpy as np
import tensorflow as tf
import sys

sys.path.append('/Users/mme/projects/models/research/slim')

from nets.mobilenet import mobilenet_v3

In [3]:
is_training = tf.placeholder(dtype=tf.bool, shape=())

x = tf.placeholder(dtype=tf.float32, shape=(None, 10, 224, 224, 3))
x_flat = tf.reshape(x, (-1, 224, 224, 3))

y = tf.placeholder(dtype=tf.float32, shape=(None, 5))

with tf.variable_scope('mobilenet'):
    with tf.contrib.slim.arg_scope(mobilenet_v3.training_scope()):
        logits, endpoints = mobilenet_v3.mobilenet(x_flat, num_classes=0, is_training=is_training)
    #net, endpoints = mobilenet_v3.mobilenet(x_flat, num_classes=0, is_training=is_training)
    #logits, endpoints = resnet_v1.resnet_v1_50(x_flat, is_training=is_training)
    #logits, endpoints = mobilenet_v1.mobilenet_v1(x_flat)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `layer.__call__` method instead.


In [4]:
logits

<tf.Tensor 'mobilenet/MobilenetV3/Logits/AvgPool2D:0' shape=(?, 1, 1, 1280) dtype=float32>

In [6]:
features = tf.squeeze(logits, [1, 2])
hidden = tf.layers.dense(features, 50, activation=tf.nn.relu)


reshaped = tf.reshape(hidden, (-1, 10, 50))
concat = tf.concat([tf.reduce_mean(reshaped, axis=1), tf.reduce_max(reshaped, axis=1)], axis=1)

outcomes = tf.layers.dense(concat, 5, activation=None)

Instructions for updating:
Use keras.layers.Dense instead.


In [7]:
mobilenet_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='mobilenet')
print(mobilenet_vars)

[<tf.Variable 'mobilenet/MobilenetV3/Conv/weights:0' shape=(3, 3, 3, 16) dtype=float32_ref>, <tf.Variable 'mobilenet/MobilenetV3/Conv/BatchNorm/gamma:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'mobilenet/MobilenetV3/Conv/BatchNorm/beta:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'mobilenet/MobilenetV3/Conv/BatchNorm/moving_mean:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'mobilenet/MobilenetV3/Conv/BatchNorm/moving_variance:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'mobilenet/MobilenetV3/expanded_conv/depthwise/depthwise_weights:0' shape=(3, 3, 16, 1) dtype=float32_ref>, <tf.Variable 'mobilenet/MobilenetV3/expanded_conv/depthwise/BatchNorm/gamma:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'mobilenet/MobilenetV3/expanded_conv/depthwise/BatchNorm/beta:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'mobilenet/MobilenetV3/expanded_conv/depthwise/BatchNorm/moving_mean:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'mobilenet/MobilenetV3/expanded_conv/depthwise/Batch

In [8]:
saver = tf.train.Saver(mobilenet_vars)

In [9]:
CHECKPOINT_FILE = '../v3-small_224_1.0_float/pristine/model.ckpt'

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, CHECKPOINT_FILE)

ValueError: The passed save_path is not a valid checkpoint: ../v3-small_224_1.0_float/pristine/model.ckpt